Get the dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

c:\Users\allen\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\allen\.cache\kagglehub\datasets\msambare\fer2013\versions\1


Extract HOG Features from Training Images

In [2]:
import os
import cv2
import numpy as np
from skimage.feature import hog

In [3]:
path = '../train'
emotions = os.listdir(path)
print("Emotions found:", emotions)

Emotions found: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [4]:
features = []
labels = []

In [5]:
trail_path = os.path.join(path, emotions[0])
print("First emotion path:", trail_path)

First emotion path: ../train\angry


In [6]:
features = []
labels = []

for emotion in emotions:
    emotion_path = os.path.join(path, emotion)
    for img_name in os.listdir(emotion_path):
        img_path = os.path.join(emotion_path, img_name)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (48, 48))
        
        hog_feat = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        features.append(hog_feat)  # ✅ no tolist here
        labels.append(emotion)

features = np.array(features, dtype='float32')  # ✅ this will now be (900, ~2916)
labels = np.array(labels)


In [7]:
# import matplotlib.pyplot as plt
# trail_path_for_img = os.path.join(trail_path, 'Training_3908.jpg')
# img = cv2.imread(trail_path_for_img, cv2.IMREAD_GRAYSCALE)
# features, hog_image = hog(img,
#                           pixels_per_cell=(8, 8),
#                           cells_per_block=(2, 2),
#                           visualize=True,
#                           feature_vector=True)

# plt.figure(figsize=(8, 4))
# plt.subplot(1, 2, 1)
# plt.imshow(img, cmap='gray')
# plt.title('Input image')

# plt.subplot(1, 2, 2)
# plt.title('HOG Visualization')
# plt.imshow(hog_image, cmap='gray')

# plt.tight_layout()
# plt.show()



In [8]:
features.shape

(28709, 900)

In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Optional: View the mapping
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_map)


Label Mapping: {np.str_('angry'): np.int64(0), np.str_('disgust'): np.int64(1), np.str_('fear'): np.int64(2), np.str_('happy'): np.int64(3), np.str_('neutral'): np.int64(4), np.str_('sad'): np.int64(5), np.str_('surprise'): np.int64(6)}


Train the model with the available images

In [10]:
from sklearn.svm import SVC
# Create and train the SVM model
model = SVC(kernel='linear', )
model.fit(features, encoded_labels)     

SVC(kernel='linear')

In [12]:
from sklearn.metrics import accuracy_score, classification_report

test_dir = "../test"
test_features = []
test_labels = []

for emotion in os.listdir(test_dir):
    emotion_path = os.path.join(test_dir, emotion)
    for img_name in os.listdir(emotion_path):
        img_path = os.path.join(emotion_path, img_name)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (48, 48))
        hog_feat = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        test_features.append(hog_feat)
        test_labels.append(emotion)

test_features = np.array(test_features, dtype='float32')
test_labels = label_encoder.transform(test_labels)  # encode using same encoder used during training


In [13]:
predictions = model.predict(test_features)

accuracy = accuracy_score(test_labels, predictions)
report = classification_report(test_labels, predictions, target_names=label_encoder.classes_)

print("✅ Accuracy:", accuracy)
print("📊 Classification Report:\n", report)


✅ Accuracy: 0.44483142936751185
📊 Classification Report:
               precision    recall  f1-score   support

       angry       0.32      0.30      0.31       958
     disgust       0.49      0.22      0.30       111
        fear       0.31      0.24      0.27      1024
       happy       0.56      0.73      0.64      1774
     neutral       0.41      0.44      0.43      1233
         sad       0.33      0.29      0.31      1247
    surprise       0.59      0.52      0.55       831

    accuracy                           0.44      7178
   macro avg       0.43      0.39      0.40      7178
weighted avg       0.43      0.44      0.43      7178



In [14]:
import joblib

joblib.dump(model, "emotion_svm_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
print("✅ Model and label encoder saved.")


✅ Model and label encoder saved.
